<a href="https://colab.research.google.com/github/ykitaguchi77/ImageProcessing/blob/master/Split_dataset_for_crossvalidation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Split datasets for cross validation
データセットを10分割し、そのうち1つをtestセットに、残りの合計をtraining+validationセットに分割するスクリプト


In [1]:
import random
import glob
import os
import shutil
import numpy as np

from google.colab import drive
drive.mount('/content/drive')


'''
-----dst_data[0]------dst_train[0]
  |                |--dst_val[0]
  |                |--dst_test[0]
  |
  |--dst_data[1]------dst_train[1]
  |                |--dst_val[1]
  |                |--dst_test[1]
  ...
  |--dst_data[9]------dst_train[9]
                   |--dst_val[9]
                   |--dst_test[9]
'''

Mounted at /content/drive


'\n-----dst_data[0]------dst_train[0]\n  |                |--dst_val[0]\n  |                |--dst_test[0]\n  |\n  |--dst_data[1]------dst_train[1]\n  |                |--dst_val[1]\n  |                |--dst_test[1]\n  ...\n  |--dst_data[9]------dst_train[9]\n                   |--dst_val[9]\n                   |--dst_test[9]\n'

#**モジュール群**



In [46]:
def get_data_list(org_path, split_num):
    classes = os.listdir(org_path) #クラス名を取得
    #print(classes)

    #データの分割数を設定
    data_list = [0]*len(classes)
    k=0
    for i in classes:
        data_list = glob.glob(org_path+'/'+i+'/*')

    split_length = int(len(data_list)/split_num)
    return data_list, split_length, classes


def makefolders(dst_path, split_num, classes):
    if not os.path.exists(dst_path):  # ディレクトリがなかったら
        os.mkdir(dst_path)  # 作成したいフォルダ名を作成
        for i in range(split_num):
            os.mkdir(dst_path+'/'+str(i))
            os.mkdir(dst_path+'/'+str(i)+'/train')
            os.mkdir(dst_path+'/'+str(i)+'/val')
            os.mkdir(dst_path+'/'+str(i)+'/test')
            for j in classes:
                os.mkdir(dst_path+'/'+str(i)+'/train/'+j)
                os.mkdir(dst_path+'/'+str(i)+'/val/'+j)
                os.mkdir(dst_path+'/'+str(i)+'/test/'+j)

def split_data_list(data_list, split_num):

    split_data, dst_data, dst_train, dst_val, dst_test = [0]*split_num, [0]*split_num, [0]*split_num, [0]*split_num, [0]*split_num

    #データの分割
    split_data = list(np.array_split(data_list, split_num))

    #データセット全体と分割したデータの差分を取り、dst_dataに格納
    dst_data = [0] * split_num
    for i in range(split_num):
        dst_data[i] = [x for x in data_list if x not in split_data[i]]
        #print(dst_data[i])

    #トレーニングセット、バリデーションセット、テストセットのリスト作成
    for i in range(split_num):
        dst_train[i], dst_val[i]= list(np.array_split(dst_data[i], [int(len(dst_data[i]) * 0.8)]))  #dst_dataを、トレーニングセットとバリデーションセットに分割
        dst_test[i] = split_data[i]  #テストセット
    
    return dst_train, dst_val, dst_test

#**リスト化したデータを作成したフォルダに移動**

In [47]:
def copy_files(split_num, dst_train, dst_val, dst_test, classes):

    for i in range(split_num):
        dst_path_train = '/content/drive/My Drive/gravcont_crossvalidation/'+str(i)+'/train/'+classes
        dst_path_val = '/content/drive/My Drive/gravcont_crossvalidation/'+str(i)+'/val/'+classes
        dst_path_test = '/content/drive/My Drive/gravcont_crossvalidation/'+str(i)+'/test/'+classes

        for p in dst_train[0]:  # 選択したファイルを目的フォルダにコピー
            shutil.copy(p, dst_path_train)
            #print(p)

        for p in dst_val[0]:  # 選択したファイルを目的フォルダにコピー
            shutil.copy(p, dst_path_val)
            #print(p)    
            
        for p in dst_test[0]:  # 選択したファイルを目的フォルダにコピー
            shutil.copy(p, dst_path_test)
            #print(p)


In [48]:
def main():
    org_path = "/content/drive/My Drive/gravcont"
    dst_path = "/content/drive/My Drive/gravcont_crossvalidation"  # フォルダ名
    split_num = 10  
    data_list, split_length, classes = get_data_list(org_path, split_num)
    makefolders(dst_path, split_num, classes)

    print(classes)

    k=0
    for i in range(len(classes)):
        dst_train, dst_val, dst_test = split_data_list(data_list[k], split_num)
        copy_files(split_num, classes[k], dst_train, dst_val, dst_test)

if __name__ == "__main__":  
    main()

['grav', 'cont']
2


AxisError: ignored

#**作ったフォルダを削除したいとき**

In [45]:
dst_path = "/content/drive/My Drive/gravcont_crossvalidation"
directory = dst_path
try:
    shutil.rmtree(directory)
except FileNotFoundError:
    pass